In [2]:
!pip install --quiet datasets
!pip install --quiet emoji
!pip install --quiet torchinfo
!pip install --quiet imbalanced-learn
!pip install --quiet gdown
!pip install --quiet clean-text
!pip install --quiet accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.9 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packag

In [4]:
#torch
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, MSELoss, CrossEntropyLoss
from torch.optim import AdamW
from torch.utils import data
from torch.utils.data import Dataset as ds, DataLoader

# dataset
import datasets
from datasets import Dataset
from datasets import Sequence
from datasets import Value
from datasets import Features
from datasets import ClassLabel
from datasets import DatasetDict

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

import re, string
import emoji
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import joblib
import traceback
from tqdm import tqdm
from collections import defaultdict

In [5]:
def clean_text(text):
  text = re.sub('#', '', text) # Removing '#' hashtag
  text = re.sub('\w+:\/\/\S+', '', text) # Removing hyperlink
  text = re.sub('[^a-zA-Z]', ' ', text) # Remove punctuation
  return text.lower()

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

def create_dataset(df, text, label):
  class_names = ['non-toxic','mild', 'severe']
  data_dict = {'text':df[text], 'labels':df[label]}
  tags = ClassLabel(num_classes=3 , names=class_names)
  feature_set = Features({'text':Value(dtype='string'), 'labels':tags})

  return Dataset.from_dict(mapping = data_dict, features = feature_set)

In [7]:
def split_dataset(random_seed=42):
  np.random.seed(random_seed)
  torch.manual_seed(random_seed)
  data = pd.read_csv('/content/trim.csv')
  other_fields = ['date time', 'match', 'region', 'game time', 'slot', 'player slot']
  data = data.drop(other_fields, 1)
  class_map = {'non-toxic': 0, 'mild': 1, 'severe': 2}
  data['target'] = data.apply(lambda x: class_map[x['toxicity']], axis=1)
  data = data.drop('toxicity', 1)

  #split data, get 25% for testing
  train, test = train_test_split(data, test_size = 0.25, random_state = 0)
  #clean the data
  train.loc[:,'message'] = train['message'].apply(lambda x : clean_text(x))
  test.loc[:,'message'] = test['message'].apply(lambda x : clean_text(x))
  #split training data, get 10% for validation
  train_all_convert = train.copy()
  train_all_convert['target'] = train_all_convert['target'].astype(str)
  train_all_convert.rename(columns={'message':'prompt', 'target':'completion'}, inplace=True)
  train_all_convert.to_json("train_all_pandas.jsonl", orient='records', lines=True)

  train, validation = train_test_split(train, test_size=0.1, random_state=random_seed, stratify=train['target'])
  train.to_csv('/content/train.csv', index=False)
  validation.to_csv('/content/validation.csv', index=False)
  test.to_csv('/content/test.csv', index=False)



In [8]:
split_dataset()

<ipython-input-7-c5ecc2a10ff9>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop(other_fields, 1)
<ipython-input-7-c5ecc2a10ff9>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('toxicity', 1)
